<a href="https://colab.research.google.com/github/SidiMohamedAJ/Machine_learning-and-Deep_learning-Projects/blob/main/Moroccan_Sentiment_Analysis_using_a_fine_tune_bert_model_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

In [4]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [5]:
!pip install PyArabic
!pip install Tashaphyne
!pip install Arabic-Stopwords


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 3.6 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import nltk

nltk.download('stopwords')
nltk.download("punkt")
from nltk.corpus import stopwords
from pyarabic.araby import tokenize, is_arabicrange, strip_tashkeel
from pyarabic import trans
import arabicstopwords.arabicstopwords as stp
import string
import re
from gensim import corpora, models
import http.client
import json
import requests
from tashaphyne.stemming import ArabicLightStemmer

#Strip vowels from a text, include Shadda
from pyarabic.araby import strip_tashkeel
#Standardize the Hamzat into one form of hamza, replace Madda by hamza and alef.
import pyarabic.araby as araby

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
from itertools import groupby

**Preprocessing**

In [8]:
class preprocessing:

  arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
  english_punctuations = string.punctuation
  punctuations_list = arabic_punctuations + english_punctuations
  stop_words = stopwords.words('arabic')
  stop_words.append(['ان','او','الى','اي','اما','شي','الا','يا'])
  ArListem = ArabicLightStemmer()

  def __init__(self, stemming = None, del_punctiation = True, del_emoji = True, del_consecutive = True, del_stopword = True):
    self.stemming = stemming
    self.del_punctiation = del_punctiation
    self.del_emoji = del_emoji
    self.del_consecutive = del_consecutive
    self.del_stopword = del_stopword


  #Flatten
  def __flatten(self, t):
    return [item for sublist in t for item in sublist]

  #remove punctiation
  def __remove_punctuations(self, text):
    translator = str.maketrans('', '', self.punctuations_list)
    return text.translate(translator)

  def del_punc(self, elements):
    if self.del_punctiation == True:
      wordsForTf = []
      for element in elements:
        word = self.__remove_punctuations(element)
        if word != '':
          wordsForTf.append(word)
      return wordsForTf
    else:
      return elements

  #remove consecutive
  def remove_consecutive(self, text:list):
    if self.del_consecutive == True:
      comment=list()
      for word in text:
          if len(word)==1:
              continue
          groups = groupby(word)
          new_text=''
          result = [(label, sum(1 for _ in group)) for label, group in groups]
          for letter in result:
              if letter[1]>2:
                  new_text+=letter[0]
              else:
                  if letter[1]==2:
                      for i in range(2):
                          new_text+=letter[0]
                  else:
                      new_text+=letter[0]
          comment.append(new_text)
      return comment
    else:
      return text

  #remove Emoji
  def deEmoji(self, text):
    if self.del_emoji == True:
      regrex_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F"
          u"\U0001F300-\U0001F5FF"
          u"\U0001F680-\U0001F6FF"
          u"\U0001F1E0-\U0001F1FF"
                            "]+", flags = re.UNICODE)
      return regrex_pattern.sub(r'',text)
    else:
      return text

  #remove Stop Words
  def del_StopWords(self, wordss,hamza_normalize=False):

    if self.del_stopword == True:
      if hamza_normalize == True:
        stop_words_normalize=[self.normalize_hamza(word) for word in self.stop_words]
        return [ [word for word in comment if word not in stop_words_normalize] for comment in wordss ]

      wordsWstop = []
      wordsWstopT = []
      for liste in wordss:
        wordsWstop.append([element for element in liste if stp.is_stop(element) == False])

      for liste in wordsWstop:
        wordsWstopT.append([element for element in liste if element not in self.stop_words])
      return wordsWstopT
    else:
      return wordss


  def token(self, textSEmoji):
    words = []
    for comment in textSEmoji:
      words.append(tokenize(comment, conditions=is_arabicrange, morphs=strip_tashkeel))

    return words


  def stemLineLight(self, tokens):
    StemLight = []
    for token in tokens:
      stem = self.ArListem.light_stem(token)
      StemLight.append(self.ArListem.get_stem())
    return StemLight

  def uniques(self,data):
        #finding unique
    unique = []
    for doc in data:
      for word in doc:
        if word not in unique:
            unique.append(word)
    return len(unique)


  def remove_length2(self,data:list,length=2):
    return [[word for word in comment if len(word) >length ] for comment in data]


  def normalize_hamza(self,data,method="tasheel"):
    if type(data)==str:
      return araby.normalize_hamza(data, method=method)

    return [araby.normalize_hamza(comment, method=method) for comment in data]

  def preprocess(self, data):
    #list of text data
    textSEmoji = np.array([self.deEmoji(text) for text in data])

    wordss = []
    for word in self.token(textSEmoji):
      # word is vector of words
      wordss.append(self.remove_consecutive(word))

    wordss=self.del_StopWords(wordss)


    stemmi = [self.stemLineLight(tokens) for tokens in wordss]
    stemmi = wordss

    wordsWstop = self.del_StopWords(stemmi)

    wordsForTf = np.array([self.del_punc(word) for word in wordsWstop], dtype=object)

    return wordsForTf

**Load Data**

In [9]:
path='/content/drive/MyDrive/Colab Notebooks/sentiment-analysis-on-moroccan-arabic-dialect/train.csv'

data = pd.read_csv(path)
IDs=data['ID'].to_numpy()
comments = data['comment'].to_numpy()
labeles =  data['label'].to_numpy()
data.head()

,ID,comment,label
0,1,انا اوصي من هذا المنبر الكل للتوجه الى مراكز ا...,1
1,2,\nهناك الكثير لا يفهم المقصود كورونا ليست صعبة...,1
2,3,الحمد لله ارقام مقبولة مقارنة بدول المنطقة لول...,1
3,4,انا شخصيا أؤيد ما فرضته السلطات من ضرورة الإدل...,1
4,5,\nنفس الشئ في مدينة برشيد مراكز التلقيح مغلقة ...,1


In [10]:
prepro = preprocessing(stemming='light')
textPre = prepro.preprocess(comments)

X_data = [' '.join(doc) for doc in textPre]

y_data = labeles

**Fine Tune a bert model**

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")
bert_model = AutoModelForSequenceClassification.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [12]:
from sklearn.model_selection import train_test_split
X = X_data
y = y_data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [13]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:  # Check if self.labels is not None
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


In [14]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [15]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(output_dir="Fine-tuned-msa-bert", evaluation_strategy="epoch")

trainer = Trainer(
    model=bert_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [18]:
# Train pre-trained model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.511459,0.742188
2,No log,0.563912,0.744792


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.511459,0.742188
2,No log,0.563912,0.744792
3,0.458500,1.015232,0.770833


TrainOutput(global_step=576, training_loss=0.41407597561677295, metrics={'train_runtime': 12594.0869, 'train_samples_per_second': 0.366, 'train_steps_per_second': 0.046, 'total_flos': 513860504822784.0, 'train_loss': 0.41407597561677295, 'epoch': 3.0})

In [19]:
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/sentiment-analysis-on-moroccan-arabic-dialect/fineTunedBertModel_MSA')

**use the Fined tuned Model**

In [20]:
from transformers import AutoModel
Ft_model = AutoModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/sentiment-analysis-on-moroccan-arabic-dialect/fineTunedBertModel_MSA")

In [22]:
texts = X_data
labels = y_data

tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


**extract embedding**

In [23]:
def sent_to_embd(sentences, model, tokenizer):
    embd = list()
    for sentence in sentences:
        input_ids = tokenizer.encode(sentence, add_special_tokens=True)
        input_ids = torch.tensor([input_ids])
        with torch.no_grad():
            last_hidden_states = model(input_ids)[0]
        last_hidden_states = last_hidden_states.mean(1)
        embd.append(last_hidden_states)
    return np.vstack(embd)

In [24]:
aggregated_embeddings = sent_to_embd(X_data, Ft_model, tokenizer)
aggregated_embeddings.shape

(1920, 768)

**pass the embedding to Machine Learning Models**

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(aggregated_embeddings, labels, test_size=0.1, random_state=42)

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier


models = {"Logistic Regression": LogisticRegression(),
          "KNN": KNeighborsClassifier(6),
          "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=4, random_state=42),
          "SVC": SVC(kernel = 'rbf'),
          "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, max_depth=1, random_state=0),
          "MLP Classifier": MLPClassifier(hidden_layer_sizes=(64,), max_iter=1000, random_state=42)
         }

def fit_and_score(models, X_train, X_test, y_train, y_test):
    np.random.seed(42)
    model_scores = {}
    best_model = None
    best_score = 0
    model_name = None

    for name, model in models.items():
        model.fit(X_train, y_train)
        model_scores[name] = model.score(X_test, y_test)
        if model_scores[name] >= best_score:
          best_score = model_scores[name]
          best_model = model
          model_name = name
    return model_scores, best_model, best_score, model_name

In [27]:
model_scores, model, score, name = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_test,
                             y_train=y_train,
                             y_test=y_test)
print(model_scores)
print("model : ", name, " score : ",score)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Logistic Regression': 0.9635416666666666, 'KNN': 0.9635416666666666, 'Random Forest': 0.9791666666666666, 'SVC': 0.9739583333333334, 'Gradient Boosting': 0.9791666666666666, 'MLP Classifier': 0.96875}
model :  Gradient Boosting  score :  0.9791666666666666


In [28]:
import pickle

In [29]:
filename = "/content/drive/MyDrive/Colab Notebooks/sentiment-analysis-on-moroccan-arabic-dialect/model_version2.pickle"

# save model
pickle.dump(model, open(filename, "wb"))

**Load Model**

In [30]:
filename = "/content/drive/MyDrive/Colab Notebooks/sentiment-analysis-on-moroccan-arabic-dialect/model_version2.pickle"

loaded_model = pickle.load(open(filename, "rb"))

In [31]:
#y_predicted = loaded_model.predict(X)
loaded_model.score(X_test, y_test)

0.9791666666666666

**Submission**

In [32]:
data_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment-analysis-on-moroccan-arabic-dialect/test_stage1.csv')

sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment-analysis-on-moroccan-arabic-dialect/sample_submission.csv')

In [33]:
sample_submission.head()

,ID,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [34]:
data_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sentiment-analysis-on-moroccan-arabic-dialect/test_stage1.csv')
IDs_test=data_test['ID'].to_numpy()
comments_test = data_test['comment'].to_numpy()

submission = data_test['comment']

prepro = preprocessing(stemming='light')

textPre_dev = prepro.preprocess(submission)
submission = [' '.join(doc) for doc in textPre_dev]

submission_embeddings = sent_to_embd(submission, Ft_model, tokenizer)
submission_embeddings.shape

(240, 768)

In [35]:
predictions = loaded_model.predict(submission_embeddings)
sample_submission['label'] = predictions

In [36]:
sample_submission

,ID,label
0,1,1
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
235,236,0
236,237,0
237,238,1
238,239,1


In [37]:
sample_submission.to_csv("/content/drive/MyDrive/Colab Notebooks/sentiment-analysis-on-moroccan-arabic-dialect/submission.csv", index=False)